In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tools.metrics import diversity, uniqueness
from tools import add_columns_to_table

import tlc
import numpy as np

In [ ]:
table = tlc.Table.from_names("reduced_0000", "CIFAR-10-val", "3LC Tutorials")

In [ ]:
table.columns

In [ ]:
label_column_pa = table.get_column("Label")
label_column = np.array(label_column_pa.to_numpy(zero_copy_only=False).tolist(), dtype=np.int32)

In [ ]:
embeddings_column_pa = table.get_column("embedding_pacmap")
embeddings_column = np.array(embeddings_column_pa.to_numpy(zero_copy_only=False).tolist(), dtype=np.float32)

In [ ]:
diversity = diversity(label_column, embeddings_column)
uniqueness = uniqueness(label_column, embeddings_column)

In [ ]:
added_columns_table = add_columns_to_table(
    table,
    columns={
        "image_diversity": diversity,
        "image_uniqueness": uniqueness,
    },
    output_table_name="added-classification-metrics",
)